# Introduction

* Datasets:
    * https://huggingface.co/datasets/sahil2801/CodeAlpaca-20k
* Models:
    * https://huggingface.co/Qwen/Qwen1.5-0.5B

In [1]:
!pip install -U accelerate transformers trl datasets bitsandbytes peft tensorboard

  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
Using cached fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.1
    Uninstalling fsspec-2024.3.1:
      Successfully uninstalled fsspec-2024.3.1


In [2]:
# For Kaggle, if you get `TypeError: expected string or bytes-like object` when importing datasets.
!rm -r /opt/conda/lib/python3.10/site-packages/fsspec*
!pip install --force-reinstall --no-deps fsspec

  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)


In [1]:
import os
import torch

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
    BitsAndBytesConfig
)
from trl import SFTTrainer
from peft import LoraConfig

## Configuration

In [2]:
batch_size = 1
num_workers = os.cpu_count()
# max_steps = -1 for epoch-wise training.
# epochs = -1 for step-wise training.
# Both cannot be -1.
max_steps = -1
epochs = 10
bf16 = True
fp16 = False
gradient_accumulation_steps = 512
context_length = 1024
logging_steps = 50
save_steps = 50
learning_rate = 0.0001
model_name = 'Qwen/Qwen1.5-0.5B'
out_dir = 'outputs/qwen_05b_code_alpaca'

## Load Dataset 

In [3]:
dataset = load_dataset('sahil2801/CodeAlpaca-20k')

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 20022
    })
})


In [5]:
print(dataset['train'][0])

{'instruction': 'Create an array of length 5 which contains all even numbers between 1 and 10.', 'input': '', 'output': 'arr = [2, 4, 6, 8, 10]'}


In [6]:
full_dataset = dataset['train'].train_test_split(test_size=0.05, shuffle=True)
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']
 
print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 19020
})
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 1002
})


In [7]:
def preprocess_function(example):
    """
    Formatting function returning a list of samples (kind of necessary for SFT API).
    """
    text = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    return text

## Model

In [8]:
if bf16:
    model = AutoModelForCausalLM.from_pretrained(model_name).to(dtype=torch.bfloat16)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name)

In [9]:
print(model)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Line

## Tokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True,
    use_fast=False
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
print(tokenizer.pad_token)

<|endoftext|>


## Training

In [12]:
if max_steps == -1 and epochs > 0:
    training_args = TrainingArguments(
        output_dir=f"{out_dir}/logs",
        evaluation_strategy='epoch',
        weight_decay=0.01,
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy='epoch',
        save_strategy='epoch',
        logging_steps=logging_steps,
        num_train_epochs=epochs,
        save_total_limit=2,
        bf16=bf16,
        fp16=fp16,
        report_to='tensorboard',
        dataloader_num_workers=num_workers,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        lr_scheduler_type='constant',
    )

if max_steps > 0 and epochs == -1:
    training_args = TrainingArguments(
        output_dir=f"{out_dir}/logs",
        evaluation_strategy='steps',
        weight_decay=0.01,
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy='steps',
        save_strategy='steps',
        logging_steps=logging_steps,
        save_steps=save_steps,
        save_total_limit=2,
        bf16=bf16,
        fp16=fp16,
        report_to='tensorboard',
        max_steps=max_steps,
        dataloader_num_workers=num_workers,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        lr_scheduler_type='constant',
    )

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    max_seq_length=context_length,
    tokenizer=tokenizer,
    args=training_args,
    formatting_func=preprocess_function,
    packing=True
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/home/sovitdl/miniconda3/envs/experiments/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
dataloader = trainer.get_train_dataloader()
for i, sample in enumerate(dataloader):
    print(tokenizer.decode(sample['input_ids'][0]))
    print('#'*50)
    if i == 5:
        break

## Instruction:
Implement a method that reverses a linked list

### Input:
head = 1 -> 2 -> 3 -> 4 -> 5 -> 6 -> null

### Response:
// A linked list node 
class Node: 
    # Constructor to create a new node 
    def __init__(self, data): 
        self.data = data 
        self.next = None

# Function to reverse the linked list  
def reverse(head): 
    prev = None
    current = head
    while(current is not None): 
        next = current.next
        current.next = prev 
        prev = current 
        current = next
    head = prev 
    return head<|endoftext|>### Instruction:
Create a JS object property with a string value.

### Input:


### Response:
let obj = {
  name:"John Smith"
};<|endoftext|>### Instruction:
Validate the email address using a regular expression.

### Input:
email = "example@gmail.com"

### Response:
import re

# Validate the email address using the regex
email_pattern = re.compile(r'\S+@\S+\.\S+')
email_valid = re.match(email_pattern, email)

if email_valid:
  

In [15]:
history = trainer.train()

Epoch,Training Loss,Validation Loss
0,1.835000,1.282727
1,1.159700,0.910980
2,0.718100,0.837208
3,0.876900,0.809701
4,0.609100,0.799354
5,0.756900,0.798754
6,0.524900,0.808557
7,0.646700,0.828692
8,0.353500,0.872785


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [16]:
model.save_pretrained(f"{out_dir}/best_model")
tokenizer.save_pretrained(f"{out_dir}/best_model")

('outputs/qwen_05b_code_alpaca/best_model/tokenizer_config.json',
 'outputs/qwen_05b_code_alpaca/best_model/special_tokens_map.json',
 'outputs/qwen_05b_code_alpaca/best_model/vocab.json',
 'outputs/qwen_05b_code_alpaca/best_model/merges.txt',
 'outputs/qwen_05b_code_alpaca/best_model/added_tokens.json')

## Inference

In [17]:
from transformers import (
    AutoModelForCausalLM, 
    logging, 
    pipeline,
    AutoTokenizer
)

In [18]:
model = AutoModelForCausalLM.from_pretrained('outputs/qwen_05b_code_alpaca/best_model/')
tokenizer = AutoTokenizer.from_pretrained('outputs/qwen_05b_code_alpaca/best_model/')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
pipe = pipeline(
    task='text-generation', 
    model=model, 
    tokenizer=tokenizer, 
    max_length=512,
    device='cuda',
    eos_token_id=tokenizer.eos_token_id
)

In [20]:
# logging.set_verbosity(logging.CRITICAL)

In [21]:
prompt = """### Instruction:
Given a (N + 1) * N Matrix, assign each column of 1st row of matrix, the subsequent row of Matrix.

### Input:
test_list = [[5, 8, 10], [2, 0, 9], [5, 4, 2], [2, 3, 9]] Output : {5: [2, 0, 9], 8: [5, 4, 2], 10: [2, 3, 9]} 

### Response:
"""

In [22]:
print(prompt)

### Instruction:
Given a (N + 1) * N Matrix, assign each column of 1st row of matrix, the subsequent row of Matrix.

### Input:
test_list = [[5, 8, 10], [2, 0, 9], [5, 4, 2], [2, 3, 9]] Output : {5: [2, 0, 9], 8: [5, 4, 2], 10: [2, 3, 9]} 

### Response:



In [ ]:
result = pipe(
    prompt
)
print(result[0]['generated_text'])

Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [ ]:
prompt = """### Instruction:
Write Python code for merge sort.

### Input:


### Response:
"""

result = pipe(
    prompt
)
print(result[0]['generated_text'])

In [37]:
# !zip -r outputs outputs

/bin/bash: line 1: zip: command not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


```[130/130 28:52, Epoch 9/10]
Epoch	Training Loss	Validation Loss
0	1.205600	0.828289
1	0.730000	0.812995
2	0.584500	0.853283
3	0.474900	0.942427
4	0.385200	1.000935
5	0.311000	1.076774
6	0.262000	1.163679
7	0.218200	1.207516
8	0.190700	1.282566
9	0.135900	1.346347
```